## 7-5　ファインチューニングと性能評価

In [1]:
# !mkdir chap7
%cd ./chap7

/content/chap7


In [2]:
# !pip install transformers==4.18.0 fugashi===1.1.0 ipadic==1.0.0 pytorch-lightning==1.6.1

In [3]:
import glob
import json
import torch
import random
import numpy as np
import pytorch_lightning as pl

from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertModel

In [4]:
MODEL_NAME = 'tohoku-nlp/bert-base-japanese-whole-word-masking'

In [5]:
class BertForSequenceClassificationMultiLabel(torch.nn.Module):

  def __init__(self, model_name, num_labels):
    super().__init__()
    self.bert = BertModel.from_pretrained(model_name)
    self.linear = torch.nn.Linear(self.bert.config.hidden_size, num_labels)

  def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None):
    bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    last_hidden_state = bert_output.last_hidden_state

    averaged_hidden_state = (last_hidden_state*attention_mask.unsqueeze(-1)).sum(1) / attention_mask.sum(1, keepdim=True)
    scores = self.linear(averaged_hidden_state)
    output = {'logits': scores}

    if labels is not None:
      loss = torch.nn.BCEWithLogitsLoss()(scores, labels.float())
      output['loss'] = loss

    output = type('bert_output', (object,), output)

    return output

In [6]:
!wget https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/chABSA-dataset.zip
!unzip -o chABSA-dataset.zip

--2024-05-27 07:38:16--  https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/chABSA-dataset.zip
Resolving s3-ap-northeast-1.amazonaws.com (s3-ap-northeast-1.amazonaws.com)... 52.219.150.76, 52.219.8.164, 52.219.151.80, ...
Connecting to s3-ap-northeast-1.amazonaws.com (s3-ap-northeast-1.amazonaws.com)|52.219.150.76|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 722777 (706K) [application/zip]
Saving to: ‘chABSA-dataset.zip.2’

chABSA-dataset.zip. 100%[===================>] 705.84K  1.92MB/s    in 0.4s    

2024-05-27 07:38:16 (1.92 MB/s) - ‘chABSA-dataset.zip.2’ saved [722777/722777]

Archive:  chABSA-dataset.zip
  inflating: chABSA-dataset/.DS_Store  
  inflating: __MACOSX/chABSA-dataset/._.DS_Store  
 extracting: chABSA-dataset/.gitkeep  
  inflating: chABSA-dataset/e00008_ann.json  
  inflating: chABSA-dataset/e00017_ann.json  
  inflating: chABSA-dataset/e00024_ann.json  
  inflating: chABSA-dataset/e00026_ann.json  
  inflating: chAB

In [7]:
data = json.load(open('chABSA-dataset/e00030_ann.json'))
print(data['sentences'][0])

{'sentence_id': 0, 'sentence': '当期におけるわが国経済は、景気は緩やかな回復基調が続き、設備投資の持ち直し等を背景に企業収益は改善しているものの、海外では、資源国等を中心に不透明な状況が続き、為替が急激に変動するなど、依然として先行きが見通せない状況で推移した', 'opinions': [{'target': 'わが国経済', 'category': 'NULL#general', 'polarity': 'neutral', 'from': 6, 'to': 11}, {'target': '景気', 'category': 'NULL#general', 'polarity': 'positive', 'from': 13, 'to': 15}, {'target': '設備投資', 'category': 'NULL#general', 'polarity': 'positive', 'from': 28, 'to': 32}, {'target': '企業収益', 'category': 'NULL#general', 'polarity': 'positive', 'from': 42, 'to': 46}, {'target': '資源国等', 'category': 'NULL#general', 'polarity': 'neutral', 'from': 62, 'to': 66}, {'target': '為替', 'category': 'NULL#general', 'polarity': 'negative', 'from': 80, 'to': 82}]}


In [8]:
category_id = {'negative': 0, 'neutral': 1, 'positive': 2}

dataset = []
for file in glob.glob('chABSA-dataset/*.json'):
  data = json.load(open(file))

  for sentence in data['sentences']:
    text = sentence['sentence']
    labels = [0, 0, 0]

    for opinion in sentence['opinions']:
      labels[category_id[opinion['polarity']]] = 1
    sample = {'text': text, 'labels': labels}
    dataset.append(sample)

In [9]:
print(dataset[0])

{'text': '当連結会計年度におけるわが国経済は、雇用状況の改善が続き、緩やかな景気回復が続きました', 'labels': [0, 0, 1]}


In [10]:
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

max_length = 128
dataset_for_loader = []

for sample in dataset:
  text = sample['text']
  labels = sample['labels']
  encoding = tokenizer(text, max_length=max_length, padding='max_length', truncation=True)
  encoding['labels'] = labels
  encoding = {k: torch.tensor(v) for k, v in encoding.items()}
  dataset_for_loader.append(encoding)

random.shuffle(dataset_for_loader)
n = len(dataset_for_loader)
n_train = int(0.6*n)
n_val = int(0.2*n)
dataset_train = dataset_for_loader[:n_train]
dataset_val = dataset_for_loader[n_train:n_train+n_val]
dataset_test = dataset_for_loader[n_train+n_val:]

dataloader_train = DataLoader(dataset_train, batch_size=32, shuffle=True)
dataloader_val = DataLoader(dataset_val, batch_size=256)
dataloader_test = DataLoader(dataset_test, batch_size=256)

In [11]:
class BertForSequenceClassificationMultiLabel_pl(pl.LightningModule):

  def __init__(self, model_name, num_labels, lr):
    super().__init__()
    self.save_hyperparameters()
    self.bert_scml = BertForSequenceClassificationMultiLabel(model_name, num_labels=num_labels)

  def training_step(self, batch, batch_idx):
    output = self.bert_scml(**batch)
    loss = output.loss
    self.log('train_loss', loss)
    return loss

  def validation_step(self, batch, batch_idx):
    output = self.bert_scml(**batch)
    val_loss = output.loss
    self.log('val_loss', val_loss)

  def test_step(self, batch, batch_idx):
    labels = batch.pop('labels')
    output = self.bert_scml(**batch)
    scores = output.logits
    labels_predicted = (scores > 0).int()
    num_correct = (labels_predicted == labels).all(-1).sum().item()
    accuracy = num_correct / scores.size(0)
    self.log('accuracy', accuracy)

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='model/'
)

trainer = pl.Trainer(
    gpus=1,
    max_epochs=5,
    callbacks=[checkpoint]
)

model = BertForSequenceClassificationMultiLabel_pl(
    MODEL_NAME,
    num_labels=3,
    lr=1e-5
)

trainer.fit(model, dataloader_train, dataloader_val)
test = trainer.test(dataloaders=dataloader_test)
print(f'Accuracy: {test[0]["accuracy"]:.2f}')

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
Some weights of the model checkpoint at tohoku-nlp/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you ar

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py:1444: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/chap7/model/epoch=2-step=345-v1.ckpt
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at /content/chap7/model/epoch=2-step=345-v1.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.8987755179405212     │
└───────────────────────────┴───────────────────────────┘

Accuracy: 0.90


In [12]:
text_list = [
    '今期は売り上げが順調に推移したが、株価は低迷の一途を辿っている。',
    '昨年から黒字が減少した。',
    '今日の飲み会は楽しかった。'
    ]

best_model_path = checkpoint.best_model_path
model = BertForSequenceClassificationMultiLabel_pl.load_from_checkpoint(best_model_path)
bert_scml = model.bert_scml.cuda()

encoding = tokenizer(
    text_list,
    padding='longest',
    return_tensors='pt'
)
encoding = {k: v.cuda() for k, v in encoding.items()}

with torch.no_grad():
  output = bert_scml(**encoding)
scores = output.logits
labels_predicted = (scores > 0).int().cpu().numpy().tolist()

for text, label in zip(text_list, labels_predicted):
  print('--')
  print(f'入力: {text}')
  print(f'出力: {label}')

Some weights of the model checkpoint at tohoku-nlp/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


--
入力: 今期は売り上げが順調に推移したが、株価は低迷の一途を辿っている。
出力: [1, 0, 1]
--
入力: 昨年から黒字が減少した。
出力: [1, 0, 0]
--
入力: 今日の飲み会は楽しかった。
出力: [0, 0, 0]
